In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [2]:
page = get("https://www.pdga.com/tour/event/87634")
soup = BeautifulSoup(page.content, "html.parser")
#find each of the division results
tourny_name = soup.find('h1', attrs={'id':"page-title"}).text.replace('/','_')
tourny_date = soup.find('li', attrs={'class':"tournament-date"}).text.split(':')[1].strip()
division = soup.find_all('details')

In [5]:
division

[<details open=""><summary><h3 class="division" id="MPO">MPO · Mixed Pro Open <span class="players"> (13)</span></h3></summary>
 <div class="tooltip-templates">
 <span id="player-details-161045">
             Jayson Ernst #161045 <br/>
             Professional <br/>
             Pottstown, Pennsylvania, United States
         </span>
 </div>
 <div class="tooltip-templates">
 <span id="player-details-229624">
             Julius Reyes #229624 <br/>
             Professional <br/>
             Schnecksville, Pennsylvania, United States
         </span>
 </div>
 <div class="tooltip-templates">
 <span id="player-details-245418">
             Matt Bruder #245418 <br/>
             Professional <br/>
             Furlong, Pennsylvania, United States
         </span>
 </div>
 <div class="tooltip-templates">
 <span id="player-details-175135">
             Shea Scott #175135 <br/>
             Professional <br/>
             Harleysville, Pennsylvania, United States
         </span>
 </div>
 <

In [9]:
for div in division:
    layout_spans = div.find_all('div') #changed from span to div
    for span in layout_spans:
        #find span that has the layout details
        if span.get('id'):
            if 'layout-details' in span.get('id'):
                course_name: str = span.a.string
                course_info: str = span.text.split(' - ')[1].split(';')
                course_layout = course_info[0].strip()
                course_par = course_info[2].strip()
                course_distance = course_info[3].strip()
print(course_name)
print(course_layout)
print(course_par)
print(course_distance)

Covered Bridge Park
Upshot 18 hole Layout
Par 54
4,106 ft.


In [12]:
course_names = []
course_layouts = []
course_pars = []
course_distances = []
round_scores = []
round_ratings = []
for div in division:
    layout_spans = div.find_all('div') # changed from span to div
    for span in layout_spans:
        #find span that has the layout details
        if span.get('id'):
            if 'layout-details' in span.get('id'):
                course_name: str = span.a.string
                course_info: str = span.text.split(' - ')[1].split(';')
                course_layout = course_info[0].strip()
                course_par = course_info[2].strip()
                course_distance = course_info[3].strip()
    round_score = None
    round_rating = None
    players = span.find_next('tbody')
    for td in players.find_all('td'):
        #find round score
        if 'round' in td.get('class'):
            round_score = td.a.string
        # find round raiting
        elif 'round-rating' in td.get('class'):
            round_rating = td.string
        if all(v is not None for v in [round_score, round_rating]):
            round_scores.append(round_score)
            round_ratings.append(round_rating)
            course_names.append(course_name)
            course_layouts.append(course_layout)
            course_pars.append(course_par)
            course_distances.append(course_distance)
            round_score = None
            round_rating = None
df = pd.DataFrame(data={
    'tourny':[tourny_name]*len(course_names),
    'date':[tourny_date]*len(course_names),
    'course':course_names, 
    'layout':course_layouts,
    'par':course_pars, 
    'distance':course_distances, 
    'score':round_scores, 
    'rating':round_ratings}
)
df

,tourny,date,course,layout,par,distance,score,rating
0,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",48,958
1,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",45,997
2,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",48,958
3,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",46,984
4,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",50,933
...,...,...,...,...,...,...,...,...
121,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",70,677
122,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",68,702
123,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",65,741
124,Wehr Upshot 5,04-Jan-2025,Covered Bridge Park,Upshot 18 hole Layout,Par 54,"4,106 ft.",77,587
